# 4 Degree of Freedom Robot Arm

In [1]:
import time
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from IPython.display import display, Math

from tools import *

## Variables and settings

In [2]:
# Constants
l0 = symbols('l_0')
l2 = symbols('l_2')
l3 = symbols('l_3')
l4 = symbols('l_4')
le = symbols('l_e')

# Variables
t = symbols('t')
theta2 = Function(r"\theta_2")(t)
theta2_dot = diff(theta2, t)
theta2_ddot = diff(theta2, t, t)

theta3 = Function(r"\theta_3")(t)
theta3_dot = diff(theta3, t)
theta3_ddot = diff(theta3, t, t)

theta4 = Function(r"\theta_4")(t)
theta4_dot = diff(theta4, t)
theta4_ddot = diff(theta4, t, t)

d1 = Function('d_1')(t)
d1_dot = diff(d1, t)
d1_ddot = diff(d1, t, t)


alias = {}
# display as x dot instead of dx/dt, and ignore dependency (t)
alias.update({theta2: symbols(r"\theta_2"),
              theta2_dot: symbols(r"\dot{\theta_2}"),
              theta2_ddot: symbols(r"\ddot{\theta_2}"),
              theta3: symbols(r"\theta_3"),
              theta3_dot: symbols(r"\dot{\theta_3}"),
              theta3_ddot: symbols(r"\ddot{\theta_3}"),
              theta4: symbols(r"\theta_4"),
              theta4_dot: symbols(r"\dot{\theta_4}"),
              theta4_ddot: symbols(r"\ddot{\theta_4}"),
              d1: symbols(r"d_1"),
              d1_dot: symbols(r"\dot{d_1}"),
              d1_ddot: symbols(r"\ddot{d_1}"),}) 

# display cos(theta) as c\theta, sin(theta) as s\theta
alias.update({sin(theta2): symbols(r"s\theta_2"),
              cos(theta2): symbols(r"c\theta_2"),
              sin(theta3): symbols(r"s\theta_3"),
              cos(theta3): symbols(r"c\theta_3"),
              sin(theta4): symbols(r"s\theta_4"),
              cos(theta4): symbols(r"c\theta_4")})

## Transformation Metrics (see forward_kinematics.py)

In [3]:
R01, T01, P01 = get_trans_mat(0, 0, l0+d1, 0, return_P=True)
R12, T12, P12 = get_trans_mat(0, 0, 0, theta2, return_P=True)
R23, T23, P23 = get_trans_mat(l2, 90, 0, theta3+pi/2, return_P=True)
R34, T34, P34 = get_trans_mat(0, 90, l3, theta4, return_P=True)
R4e, T4e, P4e = get_trans_mat(-le, 0, l4, 0, return_P=True)

R10 = R01.T
R21 = R12.T
R32 = R23.T
R43 = R34.T
Re4 = R4e.T

R0e = R01 * R12 * R23 * R34 * R4e
T0e = T01 * T12 * T23 * T34 * T4e

## Velocity Metrices

In [4]:
print('Frame 0 (Base)')
W00 = zeros(3, 1)
symprint('\Omega', 0, 0)
matprint(W00, alias)
V00 = zeros(3, 1)
symprint('V', 0, 0)
matprint(V00, alias)

print('Frame 1')
W11 = R10 * W00
symprint('\Omega', 1, 1)
matprint(W11, alias)
V11 = R10 * (V00 + W00.cross(P01)) + Matrix([0, 0, d1_dot])
symprint('V', 1, 1)
matprint(V11, alias)

print('Frame 2')
W22 = R21 * W11 + Matrix([0, 0, theta2_dot])
symprint('\Omega', 2, 2)
matprint(W22, alias)
V22 = R21 * (V11 + W11.cross(P12))
symprint('V', 2, 2)
matprint(V22, alias)

print('Frame 3')
W33 = R32 * W22 + Matrix([0, 0, theta3_dot])
symprint('\Omega', 3, 3)
matprint(W33, alias)
V33 = R32 * (V22 + W22.cross(P23))
symprint('V', 3, 3)
matprint(V33, alias)

print('Frame 4')
W44 = R43 * W33 + Matrix([0, 0, theta4_dot])
W44.simplify()
symprint('\Omega', 4, 4)
matprint(W44, alias)
V44 = R43 * (V33 + W33.cross(P34))
V44.simplify()
symprint('V', 4, 4)
matprint(V44, alias)

print('Frame e (end effector)')
Wee = Re4 * W44
Wee.simplify()
symprint('\Omega', 'e', 'e')
matprint(Wee, alias)
Vee = Re4 * (V44 + W44.cross(P4e))
Vee.simplify()
symprint('V', 'e', 'e')
matprint(Vee, alias)

Frame 0 (Base)


^0\Omega_0

Matrix([
[0],
[0],
[0]])

^0V_0

Matrix([
[0],
[0],
[0]])

...

Frame e (end effector)


^e\Omega_e

Matrix([
[ \dot{\theta_2}*c\theta_3*c\theta_4 + \dot{\theta_3}*s\theta_4],
[-\dot{\theta_2}*c\theta_3*s\theta_4 + \dot{\theta_3}*c\theta_4],
[                     \dot{\theta_2}*s\theta_3 + \dot{\theta_4}]])

^eV_e

Matrix([
[                                                  -\dot{\theta_2}*s\theta_4*(c\theta_3*l_3 + l_2) + c\theta_4*(\dot{\theta_3}*l_3 + \dot{d_1}*c\theta_3) - l_4*(\dot{\theta_2}*c\theta_3*s\theta_4 - \dot{\theta_3}*c\theta_4)],
[-\dot{\theta_2}*c\theta_4*(c\theta_3*l_3 + l_2) - l_4*(\dot{\theta_2}*c\theta_3*c\theta_4 + \dot{\theta_3}*s\theta_4) - l_e*(\dot{\theta_2}*s\theta_3 + \dot{\theta_4}) - s\theta_4*(\dot{\theta_3}*l_3 + \dot{d_1}*c\theta_3)],
[                                                                                                                                     \dot{d_1}*s\theta_3 - l_e*(\dot{\theta_2}*c\theta_3*s\theta_4 - \dot{\theta_3}*c\theta_4)]])

### The Jacobian of velocities of the end effector in frame {e} is:

In [5]:
Jee = Matrix([[Vee[0].diff(d1_dot), Vee[0].diff(theta2_dot), Vee[0].diff(theta3_dot), Vee[0].diff(theta4_dot)],
              [Vee[1].diff(d1_dot), Vee[1].diff(theta2_dot), Vee[1].diff(theta3_dot), Vee[1].diff(theta4_dot)],
              [Vee[2].diff(d1_dot), Vee[2].diff(theta2_dot), Vee[2].diff(theta3_dot), Vee[2].diff(theta4_dot)],
              [Wee[0].diff(d1_dot), Wee[0].diff(theta2_dot), Wee[0].diff(theta3_dot), Wee[0].diff(theta4_dot)],
              [Wee[1].diff(d1_dot), Wee[1].diff(theta2_dot), Wee[1].diff(theta3_dot), Wee[1].diff(theta4_dot)],
              [Wee[2].diff(d1_dot), Wee[2].diff(theta2_dot), Wee[2].diff(theta3_dot), Wee[2].diff(theta4_dot)]])

symprint('J', 'e', 'e')
matprint(Jee, alias)

^eJ_e

Matrix([
[ c\theta_3*c\theta_4,                 -c\theta_3*l_4*s\theta_4 + s\theta_4*(-c\theta_3*l_3 - l_2),  c\theta_4*l_3 + c\theta_4*l_4,    0],
[-c\theta_3*s\theta_4, -c\theta_3*c\theta_4*l_4 + c\theta_4*(-c\theta_3*l_3 - l_2) - l_e*s\theta_3, -l_3*s\theta_4 - l_4*s\theta_4, -l_e],
[           s\theta_3,                                                    -c\theta_3*l_e*s\theta_4,                  c\theta_4*l_e,    0],
[                   0,                                                         c\theta_3*c\theta_4,                      s\theta_4,    0],
[                   0,                                                        -c\theta_3*s\theta_4,                      c\theta_4,    0],
[                   0,                                                                   s\theta_3,                              0,    1]])

### Therefore the Jacobian of velocities of the end effector in frame {0} is:

In [6]:
J0e = Matrix([[R0e, zeros(3)], [zeros(3), R0e]]) * Jee
symprint('J', 0, 'e')
matprint(J0e, alias)

print('After apply .simplify():')
J0e.simplify()
symprint('J', 0, 'e')
matprint(J0e, alias)

^0J_e

Matrix([
[ c\theta_2*c\theta_3*s\theta_3 + c\theta_3*c\theta_4*(-c\theta_2*c\theta_4*s\theta_3 + s\theta_2*s\theta_4) - c\theta_3*s\theta_4*(c\theta_2*s\theta_3*s\theta_4 + c\theta_4*s\theta_2),  -c\theta_2*c\theta_3**2*l_e*s\theta_4 + (-c\theta_2*c\theta_4*s\theta_3 + s\theta_2*s\theta_4)*(-c\theta_3*l_4*s\theta_4 + s\theta_4*(-c\theta_3*l_3 - l_2)) + (c\theta_2*s\theta_3*s\theta_4 + c\theta_4*s\theta_2)*(-c\theta_3*c\theta_4*l_4 + c\theta_4*(-c\theta_3*l_3 - l_2) - l_e*s\theta_3),  c\theta_2*c\theta_3*c\theta_4*l_e + (c\theta_4*l_3 + c\theta_4*l_4)*(-c\theta_2*c\theta_4*s\theta_3 + s\theta_2*s\theta_4) + (-l_3*s\theta_4 - l_4*s\theta_4)*(c\theta_2*s\theta_3*s\theta_4 + c\theta_4*s\theta_2),  -l_e*(c\theta_2*s\theta_3*s\theta_4 + c\theta_4*s\theta_2)],
[c\theta_3*c\theta_4*(-c\theta_2*s\theta_4 - c\theta_4*s\theta_2*s\theta_3) + c\theta_3*s\theta_2*s\theta_3 - c\theta_3*s\theta_4*(-c\theta_2*c\theta_4 + s\theta_2*s\theta_3*s\theta_4), -c\theta_3**2*l_e*s\theta_2*s\theta_4 + (-c\theta_

After apply .simplify():


^0J_e

Matrix([
[0, -c\theta_2*l_e*s\theta_4 - c\theta_3*l_3*s\theta_2 - c\theta_3*l_4*s\theta_2 - c\theta_4*l_e*s\theta_2*s\theta_3 - l_2*s\theta_2, c\theta_2*(c\theta_3*c\theta_4*l_e - l_3*s\theta_3 - l_4*s\theta_3), -l_e*(c\theta_2*s\theta_3*s\theta_4 + c\theta_4*s\theta_2)],
[0,  c\theta_2*c\theta_3*l_3 + c\theta_2*c\theta_3*l_4 + c\theta_2*c\theta_4*l_e*s\theta_3 + c\theta_2*l_2 - l_e*s\theta_2*s\theta_4, s\theta_2*(c\theta_3*c\theta_4*l_e - l_3*s\theta_3 - l_4*s\theta_3),  l_e*(c\theta_2*c\theta_4 - s\theta_2*s\theta_3*s\theta_4)],
[1,                                                                                                                                0,             c\theta_3*l_3 + c\theta_3*l_4 + c\theta_4*l_e*s\theta_3,                                    c\theta_3*l_e*s\theta_4],
[0,                                                                                                                                0,                                                           s\theta

In [7]:
Matrix([[R0e, zeros(3)], [zeros(3), R0e]]).subs(alias)

Matrix([
[-c\theta_2*c\theta_4*s\theta_3 + s\theta_2*s\theta_4,  c\theta_2*s\theta_3*s\theta_4 + c\theta_4*s\theta_2, c\theta_2*c\theta_3,                                                    0,                                                    0,                   0],
[-c\theta_2*s\theta_4 - c\theta_4*s\theta_2*s\theta_3, -c\theta_2*c\theta_4 + s\theta_2*s\theta_3*s\theta_4, c\theta_3*s\theta_2,                                                    0,                                                    0,                   0],
[                                 c\theta_3*c\theta_4,                                 -c\theta_3*s\theta_4,           s\theta_3,                                                    0,                                                    0,                   0],
[                                                   0,                                                    0,                   0, -c\theta_2*c\theta_4*s\theta_3 + s\theta_2*s\theta_4,  c\theta_2*s\theta_3*s\thet

## Try another method: get $^0V_e$ and $^0\Omega_e$ from $^0R_e$ and apply differentiation

In [8]:
V0e = R0e * Vee
V0e.simplify()
symprint('V', 0, 'e')
matprint(V0e, alias)

W0e = R0e * Wee
W0e.simplify()
symprint('W', 0, 'e')
matprint(W0e, alias)

^0V_e

Matrix([
[-\dot{\theta_2}*c\theta_2*l_e*s\theta_4 - \dot{\theta_2}*c\theta_3*l_3*s\theta_2 - \dot{\theta_2}*c\theta_3*l_4*s\theta_2 - \dot{\theta_2}*c\theta_4*l_e*s\theta_2*s\theta_3 - \dot{\theta_2}*l_2*s\theta_2 + \dot{\theta_3}*c\theta_2*c\theta_3*c\theta_4*l_e - \dot{\theta_3}*c\theta_2*l_3*s\theta_3 - \dot{\theta_3}*c\theta_2*l_4*s\theta_3 - \dot{\theta_4}*c\theta_2*l_e*s\theta_3*s\theta_4 - \dot{\theta_4}*c\theta_4*l_e*s\theta_2],
[ \dot{\theta_2}*c\theta_2*c\theta_3*l_3 + \dot{\theta_2}*c\theta_2*c\theta_3*l_4 + \dot{\theta_2}*c\theta_2*c\theta_4*l_e*s\theta_3 + \dot{\theta_2}*c\theta_2*l_2 - \dot{\theta_2}*l_e*s\theta_2*s\theta_4 + \dot{\theta_3}*c\theta_3*c\theta_4*l_e*s\theta_2 - \dot{\theta_3}*l_3*s\theta_2*s\theta_3 - \dot{\theta_3}*l_4*s\theta_2*s\theta_3 + \dot{\theta_4}*c\theta_2*c\theta_4*l_e - \dot{\theta_4}*l_e*s\theta_2*s\theta_3*s\theta_4],
[                                                                                                                              

^0W_e

Matrix([
[ \dot{\theta_3}*s\theta_2 + \dot{\theta_4}*c\theta_2*c\theta_3],
[-\dot{\theta_3}*c\theta_2 + \dot{\theta_4}*c\theta_3*s\theta_2],
[                     \dot{\theta_2} + \dot{\theta_4}*s\theta_3]])

In [9]:
J0e = Matrix([[V0e[0].diff(d1_dot), V0e[0].diff(theta2_dot), V0e[0].diff(theta3_dot), V0e[0].diff(theta4_dot)],
              [V0e[1].diff(d1_dot), V0e[1].diff(theta2_dot), V0e[1].diff(theta3_dot), V0e[1].diff(theta4_dot)],
              [V0e[2].diff(d1_dot), V0e[2].diff(theta2_dot), V0e[2].diff(theta3_dot), V0e[2].diff(theta4_dot)],
              [W0e[0].diff(d1_dot), W0e[0].diff(theta2_dot), W0e[0].diff(theta3_dot), W0e[0].diff(theta4_dot)],
              [W0e[1].diff(d1_dot), W0e[1].diff(theta2_dot), W0e[1].diff(theta3_dot), W0e[1].diff(theta4_dot)],
              [W0e[2].diff(d1_dot), W0e[2].diff(theta2_dot), W0e[2].diff(theta3_dot), W0e[2].diff(theta4_dot)]])
J0e.simplify()
symprint('J', 0, 'e')
matprint(J0e, alias)

^0J_e

Matrix([
[0, -c\theta_2*l_e*s\theta_4 - c\theta_3*l_3*s\theta_2 - c\theta_3*l_4*s\theta_2 - c\theta_4*l_e*s\theta_2*s\theta_3 - l_2*s\theta_2, c\theta_2*(c\theta_3*c\theta_4*l_e - l_3*s\theta_3 - l_4*s\theta_3), -l_e*(c\theta_2*s\theta_3*s\theta_4 + c\theta_4*s\theta_2)],
[0,  c\theta_2*c\theta_3*l_3 + c\theta_2*c\theta_3*l_4 + c\theta_2*c\theta_4*l_e*s\theta_3 + c\theta_2*l_2 - l_e*s\theta_2*s\theta_4, s\theta_2*(c\theta_3*c\theta_4*l_e - l_3*s\theta_3 - l_4*s\theta_3),  l_e*(c\theta_2*c\theta_4 - s\theta_2*s\theta_3*s\theta_4)],
[1,                                                                                                                                0,             c\theta_3*l_3 + c\theta_3*l_4 + c\theta_4*l_e*s\theta_3,                                    c\theta_3*l_e*s\theta_4],
[0,                                                                                                                                0,                                                           s\theta

### Same results obtained!